In [2]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from datetime import timedelta as td

import os

# Получить путь к текущему .ipynb-файлу
BASE_DIR = os.path.abspath(".")



# Проверка
print("Текущая рабочая директория:", os.getcwd(), '\nОттуда буду брать данные, там должна быть папка data, а в ней данные по годам')

path = BASE_DIR + '\\data'

list_with_files= os.listdir(path)

Текущая рабочая директория: c:\Users\vasil\YandexDisk\project\LK_and_commets 
Оттуда буду брать данные, там должна быть папка data, а в ней данные по годам


In [25]:
cbrate_and_inflation = pd.read_excel(path + '//Инфляция и ключевая ставка Банка России.xlsx', engine='openpyxl').set_axis(['date', 'key_rate', 'infl', 'target'], axis = 1)
cbrate_and_inflation['date'] = [i.split('.')[0] + '.' + i.split('.')[1] + '0' if  i.split('.')[1] == '202' else i for i in cbrate_and_inflation['date'].astype(str)]
cbrate_and_inflation['date'] = [dt.strptime(i, '%m.%Y') for i in cbrate_and_inflation['date'].astype(str)]
cbrate_and_inflation['date'] = cbrate_and_inflation['date'].dt.strftime('%Y-%m')

exchange_rate_data = pd.read_excel(path + '//exchange_rate.xlsx', engine='openpyxl').rename(columns={'Unnamed: 0':'name'}).set_index('name').rename_axis(None, axis=0)

ru_months = {
    '01':'Янв',
    '02':'Фев',
    '03':'Мар',
    '04':'Апр',
    '05':'Май',
    '06':'Июн',
    '07':'Июл',
    '08':'Авг',
    '09':'Сен',
    '10':'Окт',
    '11':'Ноя',
    '12':'Дек',
}
ru_months_r = {v:k for k, v in ru_months.items()}
dollar_m = exchange_rate_data.iloc[[
    0, 1, exchange_rate_data.index.to_list().index('Средний номинальный курс доллара США к рублю за период')
]].set_axis(['year', 'month', 'dollar_m'], axis = 0).T
dollar_m['date'] = [dt.strptime('-'.join([str(y), ru_months_r[m]]), "%Y-%m") for y, m in zip(dollar_m['year'], dollar_m['month'])]
dollar_m['date'] = dollar_m['date'].dt.strftime('%Y-%m')
dollar_m = dollar_m.drop(['month', 'year'], axis = 1).reset_index(drop = True)


monetary_agg = pd.read_excel(path + '//monetary_agg.xlsx', engine='openpyxl').rename(columns={'Денежные агрегаты*, млрд руб.':'name'})\
    .set_index('name').rename_axis(None, axis=0)
m_agg = monetary_agg.loc[[
    'Денежный агрегат М0', 'Денежный агрегат М1', 'Денежный агрегат М2'
]].set_axis(['m0', 'm1', 'm2']).T.reset_index().rename(columns = {'index':'date'})

m_agg['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in m_agg['date']]

credits = pd.read_excel(path + '//Баланс кредитных организаций.xlsx', engine='openpyxl').rename(columns={'Баланс кредитных организаций, млн руб.*':'name'})\
    .set_index('name').rename_axis(None, axis=0)
credits_hh = credits.loc[[
    'Кредиты и займы, предоставленные  населению (домашним хозяйствам)'
]].set_axis(['credits_hh']).T.reset_index().rename(columns = {'index':'date'})
credits_hh['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in credits_hh['date']]


pmi = pd.read_excel(path + '//Бюллетень О чем говорят тренды.xlsx', sheet_name=9, engine='openpyxl').iloc[3:,:3]\
    .set_axis(['date', 'PMI_manufacturing', 'PMI_service'], axis = 1)\
            .reset_index(drop=True)
pmi['date'] = [pd.Timestamp(i) for i in pmi['date']]
pmi['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in pmi['date']]

ruonia = pd.read_excel(path + '//Динамика ставки RUONIA.xlsx', engine='openpyxl').set_axis(['date', 'ruonia'], axis = 1)
ruonia['date'] = [pd.Timestamp(i) for i in ruonia['date']]
ruonia_m = ruonia.copy(deep = True)
ruonia_m['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in ruonia_m['date']]
ruonia_m = ruonia_m.groupby('date').mean().reset_index(drop=False)


inf_exp_all_data = pd.read_excel(path + '//Наблюдаемая_и_ожидаемая_инфляция.xlsx', engine='openpyxl', sheet_name = 'Данные за все годы')\
                    .rename(columns={'Данные в таблице приводятся в % от всех опрошенных, если не указано иное.':'name'}).set_index('name').rename_axis(None, axis=0)

inf_exp_all_data = inf_exp_all_data.rename(index = {inf_exp_all_data.index.to_list()[0]:'date'})
inf_exp = inf_exp_all_data.loc[[
    'date', 'ожидаемая инфляция среди тех, кто имеет сбережения (в %)', 'ожидаемая инфляция среди тех, кто не имеет сбережений (в %)' 
]].iloc[[0,4, 5]].T.reset_index(drop = True).set_axis(['date', 'pi_e_ws', 'pi_e_wos'], axis = 1)
inf_exp['date'] = [pd.Timestamp(i) for i in inf_exp['date']]
inf_exp['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in inf_exp['date']]

roisfix = pd.read_excel(path + '//Ставка ROISfix (RUONIA Overnight Interest Rate Swap).xlsx', engine='openpyxl').iloc[1:, :]\
    .set_axis(['date', 'ruonia_1w', 'ruonia_2w', 'ruonia_1m', 'ruonia_2m', 'ruonia_3m', 'ruonia_6m', 'ruonia_1y', 'ruonia_2y'], axis = 1).fillna(0.0)
roisfix['date'] = [pd.Timestamp(dt.strptime(str(i).split(" ")[0], "%d-%m-%Y")) for i in roisfix['date']]
roisfix.iloc[:, 1:] = roisfix.iloc[:, 1:].replace('--', '0', regex=True).replace(',', '.', regex=True).astype(float)
roisfix_m = roisfix.copy(deep = True)
roisfix_m['date'] = [dt.strftime(i, "%Y-%m") for i in roisfix_m['date']]
roisfix_m = roisfix_m.groupby('date').mean().reset_index(drop=False)


brent = pd.read_excel(path + f'//Фьючерс на нефть Brent.xlsx').iloc[:, :2].set_axis(['date', 'price_brent'], axis = 1)
brent['date'] = [pd.Timestamp(i) for i in brent['date']]
brent_m = brent.copy(deep = True)
brent_m['date'] = [dt.strftime(dt.strptime(str(i).split(" ")[0], "%Y-%m-%d"), "%Y-%m") for i in brent_m['date']]
brent_m = brent_m.groupby('date').mean().reset_index(drop=False)


zcyc = pd.read_csv(path + f'//zcyc_1.csv', sep = ';').reset_index().set_axis(['date', 'time', f'rate_1'], axis = 1).iloc[1:, :]
temp_df = pd.read_csv(path + f'//zcyc_05.csv', sep = ';').reset_index().set_axis(['date', 'time', f'rate_05'], axis = 1).iloc[1:, [0,2]]
zcyc = zcyc.merge(temp_df, how = 'inner', on = 'date')
temp_df = pd.read_csv(path + f'//zcyc_025.csv', sep = ';').reset_index().set_axis(['date', 'time', f'rate_025'], axis = 1).iloc[1:, [0,2]]
zcyc = zcyc.merge(temp_df, how = 'inner', on = 'date').drop(columns=['time'])
zcyc['date'] = [pd.Timestamp(dt.strptime(str(i).split(" ")[0], "%d.%m.%Y")) for i in zcyc['date']]
zcyc.iloc[:, 1:] = zcyc.iloc[:, 1:].replace(',', '.', regex=True).astype(float)

zcyc_m = zcyc.copy(deep=True)

zcyc_m['date'] = zcyc_m['date'].dt.strftime('%Y-%m')
zcyc_m = zcyc_m.groupby('date').mean().reset_index(drop=False)

inf_us = pd.read_excel(path + '//us_infl.xlsx', sheet_name='Monthly', engine='openpyxl')
inf_us = inf_us.rename(columns= {'observation_date':'date', 'MEDCPIM158SFRBCLE':'inf_us'}).copy(deep = True)
inf_us['date'] = inf_us['date'].dt.strftime("%Y-%m")
inf_us['inf_us'] = inf_us['inf_us']/1200+1
inf_us['inf_us_cum'] = [np.prod(inf_us['inf_us'].iloc[:i+1].to_numpy()) for i in range(inf_us.shape[0])]

igrea = pd.read_excel(path + f'\\igrea.xlsx', engine='openpyxl').set_axis(['date', 'igrea'], axis = 1)
igrea['date'] = igrea['date'].dt.strftime('%Y-%m')

ipc = pd.read_excel(path + f'\\indicators_cpd.xlsx', engine='openpyxl')
ipc = ipc.T.iloc[1:, [0, 52, 53]].reset_index(drop=False).set_axis(['date', 'all_items', 'ru_cpi', 'ru_cpi_wo_servises'], axis = 1)
ipc['date'] = ipc['date'].dt.strftime("%Y-%m")


c:\Users\vasil\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
monthly_data = cbrate_and_inflation.merge(dollar_m, how = 'outer', on = 'date')\
    .merge(m_agg, how = 'outer', on = 'date')\
        .merge(inf_exp, how = 'outer', on = 'date')\
            .merge(credits_hh, how = 'outer', on = 'date')\
                .merge(pmi, how = 'outer', on = 'date')\
                    .merge(brent_m, how = 'outer', on = 'date')\
                        .merge(roisfix_m, how = 'outer', on = 'date')\
                            .merge(zcyc_m, how = 'outer', on = 'date')\
                                .merge(inf_us, how = 'outer', on = 'date')\
                                    .merge(igrea, how = "outer", on = 'date')\
                                        .merge(ipc, how = "outer", on = 'date').sort_values('date')

In [27]:
monthly_data

,date,key_rate,infl,target,dollar_m,m0,m1,m2,pi_e_ws,pi_e_wos,...,ruonia_2y,rate_1,rate_05,rate_025,inf_us,inf_us_cum,igrea,all_items,ru_cpi,ru_cpi_wo_servises
0,1968-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-13.059807,NaN,NaN,NaN
1,1968-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-9.735306,NaN,NaN,NaN
2,1968-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-11.568010,NaN,NaN,NaN
3,1968-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-11.690422,NaN,NaN,NaN
4,1968-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-10.953614,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,2025-04,21.0,10.23,4.0,83.3027,16396.8,51134.9,116499.0,12.370968,13.904651,...,18.004091,18.353182,19.907727,21.127727,1.003317,3.686231,-23.049025,0.398161,0.356462,0.310817
688,2025-05,21.0,9.88,4.0,80.456,16539.3,51031.1,117537.1,11.582353,15.034653,...,16.766316,18.022,19.348,20.429,1.002246,3.694510,-24.513850,0.359703,0.418607,0.398319
689,2025-06,20.0,9.40,4.0,78.7176,16506.8,51381.9,118219.1,11.642857,14.278723,...,16.321053,16.622,17.851,18.8315,1.003404,3.707087,-1.811927,0.335055,0.32236,0.36041
690,2025-07,18.0,8.79,4.0,78.7337,16701.5,52411.4,119094.7,11.858852,14.243119,...,14.678261,14.443478,14.975217,15.501739,1.002807,3.717491,5.753201,0.677888,0.341093,0.356627


In [231]:
daily_data = zcyc.merge(ruonia, how = 'outer', on = 'date')\
    .merge(roisfix, how = 'outer', on = 'date')\
    .merge(brent, how = 'outer', on = 'date').sort_values('date')

In [ ]:
ipc = pd.read_excel(path + f'\\indicators_cpd.xlsx', engine='openpyxl')
ipc = ipc.T.iloc[1:, [0, 52, 53]].reset_index(drop=False).set_axis(['date', 'all_items', 'ru_cpi', 'ru_cpi_wo_servises'], axis = 1)
ipc['date'] = ipc['date'].dt.strftime("%Y-%m")

In [23]:
ipc

,date,all_items,ru_cpi,ru_cpi_wo_servises
0,2002-01,1.653597,1.026335,1.008255
1,2002-02,0.928904,0.926158,0.920431
2,2002-03,0.956062,0.915182,0.905042
3,2002-04,1.156392,0.886214,0.875906
4,2002-05,1.853757,0.873257,0.875551
...,...,...,...,...
279,2025-04,0.398161,0.356462,0.310817
280,2025-05,0.359703,0.418607,0.398319
281,2025-06,0.335055,0.32236,0.36041
282,2025-07,0.677888,0.341093,0.356627


In [44]:
not_na_list = series.loc[series.notna()].index.to_list()
start_index, stop_index = not_na_list[0], not_na_list[-1]
series = series.loc[start_index:stop_index].copy()
na_list = series.loc[series.isna()].index.to_list() # type: ignore
not_na_list = series.loc[series.notna()].index.to_list()


if len(not_na_list) > 1:
    #найдем дырки без значений

    gaps = [(i, j, [k for k in range(j+1, i)]) for i, j in zip(not_na_list[1:], not_na_list[:-1]) if i-j > 1]
    fill_na = []
    for gap in gaps:

        difference = (series.iloc[gap[0]] - series.iloc[gap[1]], gap[0] - gap[1])
        if difference[1] > 2:
            print(f"Обрати внимание, что для переменной var_name есть пробел в данных\nмежду {gap[0]} и {gap[1]} наблюдениями длинной больше 2")
        fill_na += [difference[0]*i/difference[1] for i in range(1, difference[1])]

    series.iloc[na_list] = fill_na
else:
    print("Ошибка, всего одно наблюдение, это за ряд-то такой")

In [29]:
not_na_list = series.loc[series.notna()].index.to_list()
start_index, stop_index = not_na_list[0], not_na_list[-1]
series = series.loc[start_index:stop_index].copy()
na_list = series.loc[series.isna()].index.to_list() # type: ignore
not_na_list = series.loc[series.notna()].index.to_list()

In [37]:
inf_us = pd.read_excel(path + f'//{list_with_files[2]}', sheet_name='Monthly', engine='openpyxl')
inf_us = inf_us.rename(columns= {'observation_date':'date', 'MEDCPIM158SFRBCLE':'inf_us'}).copy(deep = True)
inf_us['date'] = inf_us['date'].dt.strftime("%Y-%m")
inf_us['inf_us'] = inf_us['inf_us']/1200+1
inf_us['inf_us_cum'] = [np.prod(inf_us['inf_us'].iloc[:i+1].to_numpy()) for i in range(inf_us.shape[0])]

In [51]:
list_with_files

['exchange_rate.xlsx',
 'igrea.xlsx',
 'monetary_agg.xlsx',
 'us_infl.xlsx',
 'zcyc_025.csv',
 'zcyc_05.csv',
 'zcyc_1.csv',
 'Баланс кредитных организаций.xlsx',
 'Бюллетень О чем говорят тренды.xlsx',
 'Динамика ставки RUONIA.xlsx',
 'Инфляция и ключевая ставка Банка России.xlsx',
 'Наблюдаемая_и_ожидаемая_инфляция.xlsx',
 'Ставка ROISfix (RUONIA Overnight Interest Rate Swap).xlsx',
 'Фьючерс на нефть Brent.xlsx']

In [36]:
[np.prod(inf_us['inf_us'].iloc[:i+1].to_numpy()) for i in range(inf_us.shape[0])]

[np.float64(1.00417887783321),
 np.float64(1.0083232454978666),
 np.float64(1.011358517403839),
 np.float64(1.015427595478099),
 np.float64(1.0186154680484798),
 np.float64(1.0206877544247535),
 np.float64(1.023832818476842),
 np.float64(1.028010901352497),
 np.float64(1.0321553511787536),
 np.float64(1.0362063086748787),
 np.float64(1.040422003284681),
 np.float64(1.0435664756416259),
 np.float64(1.0477863892789128),
 np.float64(1.051831174295495),
 np.float64(1.055269983947082),
 np.float64(1.0592041657151514),
 np.float64(1.0622030417497064),
 np.float64(1.066222131622918),
 np.float64(1.0704067929064154),
 np.float64(1.0745914663002043),
 np.float64(1.0786004774050746),
 np.float64(1.0817432802131066),
 np.float64(1.0840942917033853),
 np.float64(1.087274585210452),
 np.float64(1.0900606381959668),
 np.float64(1.0951506740809605),
 np.float64(1.099453099565787),
 np.float64(1.103570199896108),
 np.float64(1.1066115410293254),
 np.float64(1.1111646651904978),
 np.float64(1.113416147

In [32]:
np.prod([1,2,3])

np.int64(6)

In [34]:
inf_us['inf_us'].iloc[:-1]

0      1.004179
1      1.004127
2      1.003010
3      1.004023
4      1.003139
         ...   
506    1.003487
507    1.003317
508    1.002246
509    1.003404
510    1.002807
Name: inf_us, Length: 511, dtype: float64

In [ ]:
series.loc[series.isna()].index.to_list()



if na_list[0] == 0:
    for i in range(len(na_list)):
        if na_list[i] - na_list[i-1]>1:
            break
if i != len(na_list)-1:
    na_list = na_list[i:]

else:
    print(f"Есть начальный пропуск с первого наблюдения, его выкинул") 
    print(series.iloc[i+1:], i)
not_na_list = series.loc[series.notna()].index.to_list()


[327, 328, 329, 330, 391]

In [14]:
if len(not_na_list) > 1:
    #найдем дырки без значений

    gaps = [(i, j, [k for k in range(j+1, i)]) for i, j in zip(not_na_list[1:], not_na_list[:-1]) if i-j > 1]
    fill_na = []
    for gap in gaps:

        difference = (series.iloc[gap[0]] - series.iloc[gap[1]], gap[0] - gap[1])
        if difference[1] > 2:
            print(f"Обрати внимание, что для переменной var_name есть пробел в данных\nмежду {gap[0]} и {gap[1]} наблюдениями длинной больше 2")
        fill_na += [difference[0]*i/difference[1] for i in range(1, difference[1])]

    series.iloc[na_list] = fill_na
else:
    print("Ошибка, всего одно наблюдение, это за ряд-то такой")

print(series.iloc[i+1:], i)

Обрати внимание, что для переменной var_name есть пробел в данных
между 331 и 326 наблюдениями длинной больше 2


C:\Users\vasil\AppData\Local\Temp\ipykernel_20808\397132424.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[na_list] = fill_na


ValueError: cannot set using a list-like indexer with a different length than the value

In [15]:
fill_na

[0.2679749860186078,
 0.5359499720372156,
 0.8039249580558234,
 1.0718999440744312]

In [173]:
[dt.strptime(i, "%d-%m-%Y") for i in roisfix['date']]

[datetime.datetime(2025, 8, 20, 0, 0),
 datetime.datetime(2025, 8, 19, 0, 0),
 datetime.datetime(2025, 8, 18, 0, 0),
 datetime.datetime(2025, 8, 15, 0, 0),
 datetime.datetime(2025, 8, 14, 0, 0),
 datetime.datetime(2025, 8, 13, 0, 0),
 datetime.datetime(2025, 8, 12, 0, 0),
 datetime.datetime(2025, 8, 11, 0, 0),
 datetime.datetime(2025, 8, 8, 0, 0),
 datetime.datetime(2025, 8, 7, 0, 0),
 datetime.datetime(2025, 8, 6, 0, 0),
 datetime.datetime(2025, 8, 5, 0, 0),
 datetime.datetime(2025, 8, 4, 0, 0),
 datetime.datetime(2025, 8, 1, 0, 0),
 datetime.datetime(2025, 7, 31, 0, 0),
 datetime.datetime(2025, 7, 30, 0, 0),
 datetime.datetime(2025, 7, 29, 0, 0),
 datetime.datetime(2025, 7, 28, 0, 0),
 datetime.datetime(2025, 7, 25, 0, 0),
 datetime.datetime(2025, 7, 24, 0, 0),
 datetime.datetime(2025, 7, 23, 0, 0),
 datetime.datetime(2025, 7, 22, 0, 0),
 datetime.datetime(2025, 7, 21, 0, 0),
 datetime.datetime(2025, 7, 18, 0, 0),
 datetime.datetime(2025, 7, 17, 0, 0),
 datetime.datetime(2025, 7, 16,

In [162]:
pd.read_excel(path + f'//{list_with_files[10]}', engine='openpyxl').iloc[1:, :]\
    .set_axis(['date', 'ruonia_1w', 'ruonia_2w', 'ruonia_1m', 'ruonia_2m', 'ruonia_3m', 'ruonia_6m', 'ruonia_1y', 'ruonia_2y'], axis = 1).fillna(0.0)

,date,ruonia_1w,ruonia_2w,ruonia_1m,ruonia_2m,ruonia_3m,ruonia_6m,ruonia_1y,ruonia_2y
1,20-08-2025,17.81,17.82,17.61,16.92,16.48,15.59,14.39,13.35
2,19-08-2025,17.81,17.81,17.65,16.96,16.51,15.61,14.4,13.33
3,18-08-2025,17.8,17.81,17.72,17.03,16.55,15.6,14.4,13.31
4,15-08-2025,17.79,17.81,17.78,17.13,16.62,15.59,14.41,13.28
5,14-08-2025,17.81,17.82,17.84,17.19,16.66,15.64,14.47,13.32
...,...,...,...,...,...,...,...,...,...
2862,15-01-2014,"6,10","6,16","6,11","6,11","6,10","6,10","0,00",0.0
2863,14-01-2014,"6,16","6,22","6,20","6,15","6,14","6,10","0,00",0.0
2864,13-01-2014,"6,18","6,23","6,20","6,17","6,15","6,12","0,00",0.0
2865,10-01-2014,"5,97","6,08","6,10","6,07","6,08","6,06","0,00",0.0


In [34]:
zcyc_m['date'] = [dt.strptime(i, '%d.%M.%Y') for i in zcyc_m['date']]
zcyc_m['date'] = zcyc_m['date'].dt.strftime('%Y-%M')
zcyc['date'] = [pd.Timestamp(i) for i in zcyc['date']]
zcyc_m.iloc[:, 1:] = zcyc.iloc[:, 1:].replace(',', '.', regex=True).astype(float)
zcyc.iloc[:, 1:] = zcyc.iloc[:, 1:].replace(',', '.', regex=True).astype(float)
zcyc_m = zcyc_m.groupby('date').mean().reset_index(drop=False)

In [92]:
dollar_m['date']

0      2005-01
1      2005-02
2      2005-03
3      2005-04
4      2005-05
        ...   
242    2025-03
243    2025-04
244    2025-05
245    2025-06
246    2025-07
Name: date, Length: 247, dtype: object